# Zadania z kolokwium 2020/21
wiele rzeczy może być źle

## Zadania #1

**1.1 Dla każdego klienta podaj imię i nazwisko pracownika, który w 1997r obsłużył
najwięcej jego zamówień, podaj także liczbę tych zamówień (jeśli jest kilku
takich pracownikow to wystarczy podać imie nazwisko jednego nich). Zbiór
wynikowy powinien zawierać nazwę klienta, imię i nazwisko pracownika oraz
liczbę obsłużonych zamówien (baza northwind)**

In [ ]:
USE Northwind

In [ ]:
WITH tab AS 
    (SELECT C.CustomerID, C.CompanyName, E.EmployeeID, E.firstname, E.lastname, COUNT(*) AS cnt
    FROM Customers AS C
    INNER JOIN Orders AS O
    ON C.CustomerID = O.CustomerID
    INNER JOIN Employees AS E
    ON E.EmployeeId = O.EmployeeID
    GROUP BY C.CustomerID, C.CompanyName, E.EmployeeID, E.firstname, E.lastname)

SELECT T.CompanyName, T.FirstName, T.LastName, T.cnt
FROM tab AS T
INNER JOIN 
    (SELECT T2.CustomerID, MAX(T2.cnt) AS cnt2
    FROM tab AS T2
    GROUP BY T2.CustomerID) AS T3
ON T.CustomerID = T3.CustomerID AND T.cnt = T3.cnt2
ORDER BY 1

In [ ]:
with all_ as (
    select e.EmployeeID, c.CustomerID, e.FirstName, e.LastName, c.CompanyName
    from Orders o
             left join Employees e on e.EmployeeID = o.EmployeeID
             left join Customers C on o.CustomerID = C.CustomerID
    where year(o.OrderDate) = 1997
    ),
     count_ as (
         select EmployeeID, CustomerID, FirstName, LastName, CompanyName, count(*) as suma
         from all_
         group by EmployeeID, CustomerID, FirstName, LastName, CompanyName
     )

select c.CompanyName,
       (select top 1 suma from count_ where count_.CompanyName = c.CompanyName order by suma desc) as suma,
       (select top 1 concat(firstname, ' ', lastname)
        from count_
        where c.CompanyName = count_.CompanyName
        order by suma desc) as 'employeeName'
from Customers c

**1.2 Wypisz wszystkich członków biblioteki (imie i nazwisko, adres), liczba
aktualnie zarezerwowanych ksiazek oraz sumaryczna liczba dni zalegania z
wypozyczonymi ksiazkami. Dla kazdego czytelnika podaj informacje czy jest
dzieckiem czy doroslym, jezeli jest dzieckiem i zalegal z oddaniem ksiazek
wyswietl imie i nazwisko opiekuna prawnego.**

In [ ]:
USE Library

In [ ]:
-- to jest źle, nie bazuej na loan, tylko na loanhist

SELECT M.firstname, M.lastname,
CASE  WHEN M.member_no IN (SELECT member_no FROM juvenile) THEN A2.street ELSE A1.street END AS street,
CASE  WHEN M.member_no IN (SELECT member_no FROM juvenile) THEN A2.city ELSE A1.city END AS city,
CASE  WHEN M.member_no IN (SELECT member_no FROM juvenile) THEN A2.state ELSE A1.state END AS state,
(SELECT COUNT(*) FROM reservation AS R WHERE R.member_no = M.member_no) AS reservations,
CASE WHEN (SELECT SUM(DATEDIFF(day, LH.due_date, GETDATE())) AS cnt FROM loanhist AS LH WHERE LH.member_no = M.member_no AND LH.in_date IS NULL) IS NULL THEN 0 
    ELSE (SELECT SUM(DATEDIFF(day, LH.due_date, GETDATE())) FROM loanhist AS LH WHERE LH.member_no = M.member_no AND LH.in_date IS NULL) END AS days_over_due_date,
CASE WHEN J.member_no IS NULL THEN 'adult' ELSE 'child' END AS adult_or_child,
CASE WHEN J.member_no IS NOT NULL AND (SELECT SUM(DATEDIFF(day, LH.due_date, GETDATE())) AS cnt FROM loanhist AS LH WHERE LH.member_no = M.member_no AND LH.in_date IS NULL) > 0
    THEN (SELECT firstname FROM member WHERE member.member_no = J.adult_member_no) ELSE NULL END AS parent_name,
CASE WHEN J.member_no IS NOT NULL AND (SELECT SUM(DATEDIFF(day, LH.due_date, GETDATE())) AS cnt FROM loanhist AS LH WHERE LH.member_no = M.member_no AND LH.in_date IS NULL) > 0
    THEN (SELECT lastname FROM member WHERE member.member_no = J.adult_member_no) ELSE NULL END AS parent_name
FROM member AS M
LEFT OUTER JOIN juvenile AS J
ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
ON J.adult_member_no = A2.member_no

**1.3 Nazwy klientów którzy złożyli zamówienia w dniu 23/05/1997 oraz jeśli
obsługiwali te zamówienia pracownicy którzy mają podwłanych to ich wypisz
(imie i nazwisko)**

In [ ]:
WITH tab AS
    (SELECT E1.EmployeeID
    FROM Employees AS E1
    LEFT OUTER JOIN Employees AS E2
    ON E1.EmployeeID = E2.ReportsTo
    WHERE E2.EmployeeID IS NOT NULL)

SELECT C.CompanyName, 
    CASE WHEN (E.EmployeeID IN (SELECT EmployeeID FROM tab)) THEN E.firstName ELSE NULL END AS EmployeeFirstName,
    CASE WHEN (E.EmployeeID IN (SELECT EmployeeID FROM tab)) THEN E.lastname ELSE NULL END AS EmployeeLastName
FROM Customers AS C
INNER JOIN Orders AS O
ON C.CustomerID = O.CustomerID AND YEAR(O.OrderDate) = 1997 AND MONTH(O.OrderDate) = 5 AND DAY(O.OrderDate) = 23
INNER JOIN Employees AS E
ON O.EmployeeId = E.EmployeeID

**1.4 wypisz wszystkich członków biblioteki, którzy nigdy nic nie wypożyczyli. Stwierdz czy członek jest dzieckiem czy dorosłym, jeśli dorosłym, to wypisz liczbę jego dzieci**

In [ ]:
SELECT M.firstname, M.lastname,
    CASE WHEN J.member_no IS NULL THEN A1.street ELSE A2.street END AS street,
    CASE WHEN J.member_no IS NULL THEN A1.city ELSE A2.city END AS city,
    CASE WHEN J.member_no IS NULL THEN A1.state ELSE A2.state END AS state,
    CASE WHEN J.member_no IS NULL THEN 'adult' ELSE 'child' END AS child_or_adult,
    CASE WHEN J.member_no IS NULL THEN (SELECT COUNT(*) FROM juvenile AS J2 WHERE J2.adult_member_no = M.member_no GROUP BY J2.adult_member_no) ELSE NULL END AS number_of_children
FROM member AS M
LEFT OUTER JOIN juvenile AS J
    ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
    ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
    ON J.adult_member_no = A2.member_no
WHERE M.member_no NOT IN (SELECT DISTINCT LH.member_no FROM loanhist AS LH
UNION (SELECT L.member_no FROM loan AS L))

**1.5 Wypisz produkt który zarobił najmniej w 1996, ale zarobił cokolwiek**

In [ ]:
SELECT TOP 1 P.ProductID, P.ProductName
FROM Products AS P
INNER JOIN [Order Details] AS OD
ON P.ProductId = OD.ProductId
INNER JOIN Orders AS O
ON OD.OrderID = O.OrderID AND YEAR(O.OrderDate) = 1996
GROUP BY P.ProductID, P.ProductName
ORDER BY SUM(OD.UnitPrice * OD.Quantity * (1-OD.Discount))

**1.6  Podaj liczbę zamówień oraz wartość zamówień (uwzględnij opłatę za przesyłkę)
obsłużonych przez każdego pracownika w lutym 1997. Za datę obsłużenia
zamówienia należy uznać datę jego złożenia (orderdate). Jeśli pracownik nie
obsłużył w tym okresie żadnego zamówienia, to też powinien pojawić się na
liście (liczba obsluzonych zamówień oraz ich wartość jest w takim przypadku
równa O). Zbiór wynikowy powinien zawierać: imię i nazwisko pracownika, liczbę
obsłużonych zamówieni, wartość obsłużonych zamówień. (baza northwind)**

In [ ]:
SELECT DISTINCT E.FirstName, E.LastName,
    (SELECT COUNT(*) FROM Orders AS O1 WHERE O1.EmployeeID = E.EmployeeID AND YEAR(O1.OrderDate) = 1997 AND MONTH(O1.OrderDate) = 2) AS OrdersNumber,
    CASE
        WHEN O.EmployeeID IS NULL THEN 0
        ELSE SUM(O.Freight) + 
            (SELECT SUM(OD1.UnitPrice * OD1.Quantity * (1-OD1.Discount))
            FROM [Order Details] AS OD1 
            INNER JOIN Orders AS O2 
                ON OD1.OrderID = O2.OrderID AND O2.EmployeeID = E.EmployeeID AND YEAR(O2.OrderDate) = 1997 AND MONTH(O2.OrderDate) = 2)
    END AS TotalEarnings
FROM Employees AS E
LEFT OUTER JOIN Orders AS O
ON E.EmployeeID = O.EmployeeID AND YEAR(O.OrderDate) = 1997 AND MONTH(O.OrderDate) = 2
GROUP BY E.EmployeeID, O.EmployeeID, E.FirstName, E.LastName

**1.7 Podaj listę dzieci będących członkami biblioteki, które w dniu '2001-12-14'
zwróciły do biblioteki książkę o tytule 'Walking'. Zbiór wynikowy powinien
zawierać imię i nazwisko oraz dane adresowe dziecka. (baza library)**

In [ ]:
SELECT M.firstname, M.lastname, A.street, A.city, A.state
FROM juvenile AS J
INNER JOIN member AS M
On J.member_no = M.member_no
INNER JOIN adult AS A
ON J.adult_member_no = A.member_no
INNER JOIN loanhist AS LH
ON J.member_no = LH.member_no AND YEAR(LH.in_date) = 2001 AND MONTH(LH.in_date) = 12 AND DAY(LH.in_date) = 14
INNER JOIN title AS T
ON LH.title_no = T.title_no AND T.title = 'Walking'